# **Installing packages and importing libraries**

In [ ]:
!pip install squarify

In [ ]:
# for basic operations
import numpy as np
import pandas as pd

# for visualizations
import matplotlib.pyplot as plt
import squarify
import seaborn as sns
plt.style.use('fivethirtyeight')

# for defining path
import os

# for market basket analysis
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# **Dataset for Lab 9**

**Data Set Information:**

We have a dataset of a mall with 7500 transactions of different customers buying different items from the store. We have to find correlations between the different items in the store. so that we can know if a customer is buying apple, banana and mango. what is the next item, The customer would be interested in buying from the store.


**Problem Statement**

Market owners should want to know what customers will buy next by looking at the products they buy, so that market owners can adjust their product placement to increase product sales. This can be overcome by using the Apriori Algorithm to perform a Market Basket Analysis of the customer's buying behavior. 

The dataset can be found here in this [URL](https://drive.google.com/file/d/1DDtVOZwFQJBn0zXc69sfb2zld1PofX57/view?usp=sharing)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **Loading the dataset**

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Angraj project dmkd/01. bank-additional-full.csv',header=None, sep=";")

#Check number of rows and columns in the dataset
print("The dataset has %d rows and %d columns." % df.shape)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
pre_df = df.copy()
pre_df.drop([4,7,8,9,10,12],axis=1,inplace=True)
pre_df.head(10)

In [ ]:

pre_df.drop([0],axis=0,inplace=True)
pre_df.head(10)

In [ ]:
print("The dataset has %d rows and %d columns." % pre_df.shape)

# **Dataset Visualization**


The bigger words in the wordcloud depicts the most used job sector in the data set

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud

plt.rcParams['figure.figsize'] = (15, 15)
wordcloud = WordCloud(background_color = 'white', width = 1200,  height = 1200, max_words = 121).generate(str(pre_df[1]))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Most Popular word',fontsize = 20)
plt.show()


In [ ]:
y = df[1].value_counts().head(40000).to_frame()
y.index

In [ ]:
# plotting a tree map

plt.rcParams['figure.figsize'] = (20, 20)
color = plt.cm.cool(np.linspace(0, 1, 40))
squarify.plot(sizes = y.values, label = y.index, alpha=.8, color = color)
plt.title('Tree Map for Popular Items')
plt.axis('off')
plt.show()

# **Data Preprocessing**

There are two principal methods of representing this type of market basket data:
using either the transactional data format or the tabular data format. The transactional data format requires only two fields, an ID field and a content field,
with each record representing a single item only.

For example, the data in Table 1 could be represented using transactional data
format as shown in Table 2. In the tabular data format, each record represents
a separate transaction, with as many 0/1 flag fields as there are items. The data
from Table 2 could be represented using the tabular data format, as shown in
Figure 1.

In [ ]:
# making each customers shopping items an identical list
trans = []
for i in range(0, 41188):
    trans.append([str(pre_df.values[i,j]) for j in range(0, 15)])

# conveting it into an numpy array
trans = np.array(trans)

# checking the shape of the array
print(trans.shape)

In [ ]:
trans[:,0:10]

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
pre_df = te.fit_transform(trans)
pre_df= pd.DataFrame(pre_df, columns = te.columns_)

# getting the shape of the data
pre_df.shape

In [ ]:
pre_df.head(10)

In [ ]:
pre_df.columns

In [ ]:
pre_df.head()

# **Apriori Algorithm**

The algorithm was first proposed in 1994 by Rakesh Agrawal and Ramakrishnan Srikant. Apriori algorithm finds the most frequent itemsets or elements in a transaction database and identifies association rules between the items just like the above-mentioned example.

To construct association rules between elements or items, the algorithm considers 3 important factors which are, support, confidence and lift. Each of these factors is explained as follows:

**Support**:

The support of item I is defined as the ratio between the number of transactions containing the item I by the total number of transactions expressed as the equation specified in the Lecture slides.

**Confidence**:

This is measured by the proportion of transactions with item I1, in which item I2 also appears. The confidence between two items I1 and I2, in a transaction is defined as the total number of transactions containing both items I1 and I2 divided by the total number of transactions containing I1. ( Assume I1 as X , I2 as Y )

**Lift**:

Lift is the ratio between the confidence and support.

**Strong Rules**

Analysts may prefer rules that have either high support or high confidence, and
usually both. Strong rules are those that meet or surpass certain minimum support and confidence criteria.
For example, an analyst interested in finding which supermarket items are purchased together may set a minimum support level of 20% and a minimum confidence level of 70%. On the other hand, a fraud detection analyst or a terrorism
detection analyst would need to reduce the minimum support level to 1% or less,
since comparatively few transactions are either fraudulent or terror-related.

**Itemset**

An itemset is a set of items contained in $I$ , and a $k-itemset$ is an itemset containing
$k$ items. For example, \{beans, squash\} is a 2-itemset, and \{broccoli, green peppers,
corn\} is a 3-itemset, each from the vegetable stand set $I$. The itemset frequency is
simply the number of transactions that contain the particular itemset.

A frequent
itemset is an itemset that occurs at least a certain minimum number of times, having
itemset frequency $\geq \phi$. For example, suppose that we set $\phi = 4.$ Then itemsets that
occur more than four times are said to be frequent. We denote the set of frequent
$k$-itemsets as $F_{k}$.

## **Finding Frequent itemsets**

In [ ]:
from mlxtend.frequent_patterns import apriori

#Now, let us return the items and itemsets with at least 5% support:
apriori(pre_df, min_support = 0.01, use_colnames = True)


In [ ]:
frequent_itemsets = apriori(pre_df, min_support = 0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

In [ ]:
# getting th item sets with length = 2 and support more han 10%

frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.01) ]


## Finding Association Rules

In [ ]:
rules_mlxtend = association_rules(frequent_itemsets, metric="support", min_threshold=0.02)
rules_mlxtend.sort_values(by=["support"],ascending=False)

In [ ]:
rules_mlxtend[ (rules_mlxtend['lift'] >= 1) & (rules_mlxtend['confidence'] >= 0.3) ].head()